In [1]:
import pandas as pd
from dash import html,dcc,Input,Output,State,dash_table 
import plotly.express as px
import numpy

In [2]:
df=pd.read_csv("E:/financial_accounting.csv")
df

Date              Account         Description   Debit  Credit  \
0      2023-08-21     Accounts Payable       Transaction 1  112.56  112.56   
1      2023-08-13  Accounts Receivable       Transaction 2  775.86  775.86   
2      2023-05-11  Accounts Receivable       Transaction 3  332.81  332.81   
3      2023-02-26  Accounts Receivable       Transaction 4  203.71  203.71   
4      2023-11-06  Accounts Receivable       Transaction 5  986.26  986.26   
...           ...                  ...                 ...     ...     ...   
99995  2023-06-13            Inventory   Transaction 99996  585.75  585.75   
99996  2023-09-14     Accounts Payable   Transaction 99997  785.01  785.01   
99997  2023-02-08                 Cash   Transaction 99998  502.68  502.68   
99998  2023-07-05            Inventory   Transaction 99999  789.19  789.19   
99999  2023-07-23  Accounts Receivable  Transaction 100000  484.13  484.13   

        Category Transaction_Type Customer_Vendor Payment_Method  Reference  
0          Asset             Sale     Customer 39           Cash      67471  
1        Revenue         Purchase      Customer 3          Check      92688  
2        Revenue         Transfer     Customer 36          Check      72066  
3          Asset         Purchase     Customer 57          Check      27973  
4          Asset          Expense     Customer 92          Check      29758  
...          ...              ...             ...            ...        ...  
99995  Liability          Expense     Customer 66          Check      13348  
99996  Liability             Sale     Customer 77    Credit Card      43646  
99997    Revenue          Expense     Customer 17    Credit Card      39071  
99998      Asset         Purchase    Customer 100    Credit Card      69225  
99999      Asset          Expense     Customer 62  Bank Transfer      88567  

[100000 rows x 10 columns]

In [3]:
df.isnull().sum()

Date                0
Account             0
Description         0
Debit               0
Credit              0
Category            0
Transaction_Type    0
Customer_Vendor     0
Payment_Method      0
Reference           0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              100000 non-null  object 
 1   Account           100000 non-null  object 
 2   Description       100000 non-null  object 
 3   Debit             100000 non-null  float64
 4   Credit            100000 non-null  float64
 5   Category          100000 non-null  object 
 6   Transaction_Type  100000 non-null  object 
 7   Customer_Vendor   100000 non-null  object 
 8   Payment_Method    100000 non-null  object 
 9   Reference         100000 non-null  int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 7.6+ MB


In [5]:
df.describe()

Debit         Credit      Reference
count  100000.000000  100000.000000  100000.000000
mean      607.747368     607.747368   54859.916200
std       254.906370     254.906370   25956.740346
min         3.880000       3.880000   10000.000000
25%       401.637500     401.637500   32367.000000
50%       634.080000     634.080000   54810.500000
75%       829.990000     829.990000   77302.250000
max       999.990000     999.990000   99999.000000

In [6]:
df.duplicated().sum()

np.int64(0)

we can see that data is clean contain not any null value and no duplication found . The data describes that there is not outlier so data is fine we can move forward

In [7]:
df.Account.value_counts()

Account
Accounts Payable       25104
Inventory              25054
Accounts Receivable    25038
Cash                   24804
Name: count, dtype: int64

In [8]:
receivables = df[df["Account"] == "Accounts Receivable"]
payables = df[df["Account"] == "Accounts Payable"]

In [9]:
total_receivable = receivables["Credit"].sum()
total_payable = payables["Credit"].sum()

In [10]:
print(total_receivable ,total_payable)

15272312.940000001 15152628.73


KPI	Amount (USD)
Receivables	💵 15.27 million
Payables	💸 15.15 million

This shows:

your business is owed slightly more than it owes.

You have a positive receivables balance overall.

In [11]:
summary1=df.groupby("Category")[["Debit","Credit"]].agg("sum").reset_index()
summary1

Category        Debit       Credit
0      Asset  15270030.86  15270030.86
1    Expense  15163910.14  15163910.14
2  Liability  15072434.67  15072434.67
3    Revenue  15268361.16  15268361.16

 Insight:

Your expenses and revenues are close, suggesting a balanced cash cycle.

Total assets slightly outweigh liabilities—shows moderate financial health.

In [12]:
summary2=df.groupby("Payment_Method")[["Debit","Credit"]].agg("sum").reset_index()
summary2

Payment_Method        Debit       Credit
0  Bank Transfer  15176092.28  15176092.28
1           Cash  15147309.99  15147309.99
2          Check  15324618.14  15324618.14
3    Credit Card  15126716.42  15126716.42

Most payments are made by Check—this can help optimize payment processing strategies.

In [9]:
# financial_dashboard.py
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
import os

# Load or create data
file_path = os.path.expanduser("~/Desktop/financial_Accounting.csv")
if not os.path.exists(file_path):
    data = {
        "Date": pd.date_range(start="2024-01-01", periods=100, freq="D"),
        "Category": ["Groceries", "Utilities", "Rent", "Transport", "Entertainment"] * 20,
        "Credit": [500, 0, 0, 0, 0] * 20,
        "Debit": [0, 150, 1000, 100, 200] * 20,
        "Payment_Method": ["Cash", "Bank", "Card", "Online", "Check"] * 20
    }
    pd.DataFrame(data).to_csv(file_path, index=False)

df = pd.read_csv(file_path)
df["Date"] = pd.to_datetime(df["Date"])

# KPIs
total_credit = df["Credit"].sum()
total_debit = df["Debit"].sum()
net_balance = total_credit - total_debit

# Create Dash app
app = Dash(__name__)
app.title = "💰 Financial Dashboard"

app.layout = html.Div(style={
    "backgroundColor": "#000",
    "color": "#FFD700",
    "fontFamily": "Arial",
    "padding": "20px"
}, children=[
    html.H1("📊 Financial Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Div([
            html.H4("Total Credit"),
            html.H2(f"${total_credit:,.2f}")
        ], style={"backgroundColor": "#222", "padding": "15px", "borderRadius": "10px", "width": "30%"}),

        html.Div([
            html.H4("Total Debit"),
            html.H2(f"${total_debit:,.2f}")
        ], style={"backgroundColor": "#222", "padding": "15px", "borderRadius": "10px", "width": "30%"}),

        html.Div([
            html.H4("Net Balance"),
            html.H2(f"${net_balance:,.2f}")
        ], style={"backgroundColor": "#222", "padding": "15px", "borderRadius": "10px", "width": "30%"})
    ], style={"display": "flex", "justifyContent": "space-around", "marginBottom": "30px"}),

    dcc.Graph(figure=px.bar(df, x="Category", y="Debit", color="Category",
                            title="Debit by Category", template="plotly_dark",
                            color_discrete_sequence=["#FFD700"])),
    
    dcc.Graph(figure=px.line(df.groupby("Date")["Credit"].sum().reset_index(),
                             x="Date", y="Credit", title="Credit Over Time",
                             template="plotly_dark", markers=True, line_shape="spline",
                             color_discrete_sequence=["#FFD700"])),

    dcc.Graph(figure=px.pie(df, names="Payment_Method", values="Debit",
                            title="Payment Methods Share", template="plotly_dark",
                            color_discrete_sequence=px.colors.sequential.YlOrBr)),

    dcc.Graph(figure=px.density_heatmap(df, x="Category", y="Payment_Method", z="Debit",
                                        title="Heatmap of Debit by Category and Payment Method",
                                        template="plotly_dark", color_continuous_scale="YlOrBr")),
])

if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off
